<a href="https://colab.research.google.com/github/toddlack/tortoise-tts/blob/dev/tortoise_tts_colab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Welcome to Tortoise! 🐢🐢🐢🐢

Before you begin, I **strongly** recommend you turn on a GPU runtime.

There's a reason this is called "Tortoise" - this model takes up to a minute to perform inference for a single sentence on a GPU. Expect waits on the order of hours on a CPU.

In [ ]:
%%bash
PYTHONPATH=
pip install  condacolab

In [ ]:
import condacolab
condacolab.install()

In [ ]:
%%bash
conda create --name tortoise python=3.10
source activate tortoise
pip install tortoise-tts transformers==4.31.0 tokenizers==0.13.3
conda info --envs


In [ ]:
import locale
import os
os.environ["LC_ALL"] = "en_US.UTF-8"
locale.setlocale(locale.LC_ALL, '')
# locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
local_encoding=locale.getpreferredencoding()
print(local_encoding)

In [ ]:
%%bash
cd /content
sudo apt install python3.10-venv
python3.10 -m venv .venv
source .venv/bin/activate
pip install numba inflect psutil
pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cpu
pip install tortoise-tts transformers==4.31.0 tokenizers==0.13.3
git clone https://github.com/neonbjb/tortoise-tts.git
cd tortoise-tts
# pip install .
# pip install -r requirements.txt

In [ ]:
#first follow the instructions in the README.md file under Local Installation
%cd /content/tortoise-tts
%pwd
!pip3 install -r requirements.txt
# !python3 setup.py install

In [ ]:
%%bash
cd /content
# source activate tortoise
source .venv/bin/activate
python --version
which python3

In [ ]:
# Imports used through the rest of the notebook.
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

# This will download all the models used by Tortoise from the HF hub.
# tts = TextToSpeech()
# If you want to use deepspeed the pass use_deepspeed=True nearly 2x faster than normal
tts = TextToSpeech(use_deepspeed=False, kv_cache=True)

In [ ]:
# This is the text that will be spoken.
# text = "Joining two modalities results in a surprising increase in generalization! What would happen if we combined them all?"

# Here's something for the poetically inclined.. (set text=)

text ="""
Two roads diverged in a yellow wood,
And sorry I could not travel both
And be one traveler, long I stood
And looked down one as far as I could
To where it bent in the undergrowth;

Then took the other, as just as fair,
And having perhaps the better claim,
Because it was grassy and wanted wear;
Though as for that the passing there
Had worn them really about the same,
"""

# Pick a "preset mode" to determine quality. Options: {"ultra_fast", "fast" (default), "standard", "high_quality"}. See docs in api.py
preset = "standard"

In [ ]:
# Tortoise will attempt to mimic voices you provide. It comes pre-packaged
# with some voices you might recognize.

# Let's list all the voices available. These are just some random clips I've gathered
# from the internet as well as a few voices from the training dataset.
# Feel free to add your own clips to the voices/ folder.
%ls tortoise/voices

IPython.display.Audio('tortoise/voices/tom/1.wav')

In [ ]:
# Pick one of the voices from the output above
voice = 'train_atkins'

# Load it and send it through Tortoise.
voice_samples, conditioning_latents = load_voice(voice)
gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents,
                          preset=preset)
torchaudio.save('generated.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('generated.wav')

In [ ]:
# Tortoise can also generate speech using a random voice. The voice changes each time you execute this!
# (Note: random voices can be prone to strange utterances)
gen = tts.tts_with_preset(text, voice_samples=None, conditioning_latents=None, preset=preset)
torchaudio.save('generated.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('generated.wav')

In [ ]:
# You can also combine conditioning voices. Combining voices produces a new voice
# with traits from all the parents.
#
# Lets see what it would sound like if Picard and Kirk had a kid with a penchant for philosophy:
voice_samples, conditioning_latents = load_voices(['pat', 'william'])

gen = tts.tts_with_preset("They used to say that if man was meant to fly, he’d have wings. But he did fly. He discovered he had to.",
                          voice_samples=None, conditioning_latents=None, preset=preset)
torchaudio.save('captain_kirkard.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio('captain_kirkard.wav')

In [ ]:
# del tts  # Will break other cells, but necessary to conserve RAM if you want to run this cell.
!source /content/.venv/bin/activate
# !pip install tortoise-tts
%env LC_ALL=C.UTF-8
%env LANG=C.UTF-8
%cd /content/tortoise-tts
# Tortoise comes with some scripts that does a lot of the lifting for you. For example,
# read.py will read a text file for you.
!python3 tortoise/read.py --voice=train_atkins --textfile=tortoise/data/riding_hood.txt --preset=ultra_fast --output_path=.

# IPython.display.Audio('train_atkins/combined.wav')
# This will take awhile..